In [1]:
from numpy import genfromtxt

In [2]:
data = genfromtxt('data_1.csv', delimiter = ',', encoding = "utf8", dtype = None)

In [3]:
print(data)

[['pclass' 'age' 'gender' 'survived']
 ['1st' 'adult' 'male' 'yes']
 ['1st' 'adult' 'male' 'yes']
 ...
 ['crew' 'adult' 'female' 'no']
 ['crew' 'adult' 'female' 'no']
 ['crew' 'adult' 'female' 'no']]


In [4]:
data = data[2:]

In [5]:
print(data)

[['1st' 'adult' 'male' 'yes']
 ['1st' 'adult' 'male' 'yes']
 ['1st' 'adult' 'male' 'yes']
 ...
 ['crew' 'adult' 'female' 'no']
 ['crew' 'adult' 'female' 'no']
 ['crew' 'adult' 'female' 'no']]


In [6]:
columns = ['pclass', 'age', 'gender', 'survived']

In [7]:
def unique_values(rows, col):
    return set([row[col] for row in rows])

In [8]:
print(unique_values(data, 0))
print(unique_values(data, 2))


{'1st', '3rd', '2nd', 'crew'}
{'male', 'female'}


In [9]:
def calculate_freq(rows):
    count = {}
    for row in rows:
        label = row[-1]
        if label not in count:
            count[label] = 0
        count[label] += 1
    return count

In [10]:
calculate_freq(data)

{'yes': 710, 'no': 1490}

In [11]:
def is_numeric(val):
    return isinstance(val, int) or isinstance(val, float)

In [12]:
class Question:
    
    def __init__(self, col, value):
        self.column = col
        self.value = value
    
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
        columns[self.column], condition, str(self.value))

In [13]:
Question(1, 3)

Is age >= 3?

In [14]:
q = Question(2, 'male')

In [15]:
q

Is gender == male?

In [16]:
q = Question(0, 'crew')

In [17]:
q

Is pclass == crew?

In [18]:
ex = data[-1]
q.match(ex)

True

In [19]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [20]:
true_rows, false_rows = partition(data, q)

In [21]:
(true_rows)

[array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['crew', 'adult', 'male', 'yes'], dtype='<U8'),
 array(['c

In [22]:
def gini(rows):
    counts = calculate_freq(rows)
    impurity = 1
    for label in counts:
        prob = counts[label] / float(len(rows))
        impurity -= prob**2
    return impurity

In [23]:
def info_gain(left, right, current_gini):
    p = float(len(left)) / (len(left) + len(right))
    return current_gini - p * gini(left) - (1-p)*gini(left)


In [24]:
current_gini = gini(data)

In [25]:
current_gini

0.43714876033057853

In [26]:
true_rows, false_rows = partition(data, Question(0, '1st'))

In [27]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_gini = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) ==0 or len(false_rows) == 0:
                continue
            
            gain = info_gain(true_rows, false_rows, current_gini)
            
            if gain >= best_gain:
                best_gain, best_question = gain, question
            
    return best_gain, best_question

In [28]:
best_gain, best_question = find_best_split(data)

In [29]:
best_question

Is gender == male?

In [30]:
best_gain

0.10354322723558713

In [31]:
class Leaf:
    def __init__(self,rows):
        self.predictions = calculate_freq(rows)

In [32]:
class Decision_Node:
    def __init__(self,
                 question,
                true_branch,
                false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [33]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

In [69]:
def print_tree(node, spacing=""):
   
    if isinstance(node, Leaf):
        print (spacing + " Predict", print_leaf(node.predictions))
        return

    print (spacing + "•" + str(node.question))

    print (spacing + ' ⮕ True:')
    print_tree(node.true_branch, spacing + "   ")

    print (spacing + ' ⮕ False:')
    print_tree(node.false_branch, spacing + "   ")


In [70]:
tree = build_tree(data)

In [71]:
print_tree(tree)

•Is gender == male?
 ⮕ True:
   •Is pclass == 2nd?
    ⮕ True:
      •Is age == child?
       ⮕ True:
          Predict {'yes': '100%'}
       ⮕ False:
          Predict {'yes': '8%', 'no': '91%'}
    ⮕ False:
      •Is pclass == 3rd?
       ⮕ True:
         •Is age == adult?
          ⮕ True:
             Predict {'yes': '16%', 'no': '83%'}
          ⮕ False:
             Predict {'yes': '27%', 'no': '72%'}
       ⮕ False:
         •Is age == child?
          ⮕ True:
             Predict {'yes': '100%'}
          ⮕ False:
            •Is pclass == crew?
             ⮕ True:
                Predict {'yes': '22%', 'no': '77%'}
             ⮕ False:
                Predict {'yes': '32%', 'no': '67%'}
 ⮕ False:
   •Is pclass == 1st?
    ⮕ True:
      •Is age == child?
       ⮕ True:
          Predict {'yes': '100%'}
       ⮕ False:
          Predict {'yes': '97%', 'no': '2%'}
    ⮕ False:
      •Is pclass == 2nd?
       ⮕ True:
         •Is age == child?
          ⮕ True:
             Pre

In [38]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [39]:
classify(data[0], tree)

{'yes': 56, 'no': 118}

In [42]:
def print_leaf(count):
    total = sum(count.values()) * 1.0
    prob = {}
    for label in count.keys():
        prob[label] = str(int(count[label] / total * 100)) + "%"
    return prob


In [44]:
print_leaf(classify(data[0], tree))

{'yes': '32%', 'no': '67%'}